# Bell test for classic variable

In [5]:
import random
def setup_variables():
    
    ### Replace this section with anything you want ###
    
    r = random.random()
    
    A = r*(2/3)
    B = r*(1/3)
    
    ### End of section ###
    
    return A, B

In [6]:
def hash2bit(variable, hash_type):
    
    ### Replace this section with anything you want ###
    
    if hash_type == 'V':
        bit = (variable < 0.5)
    elif hash_type == 'H':
        bit = (variable < 0.25)
    
    bit = str(int(bit)) # Turn True or False into '1' or '0' 
    
    ### End of section ###
    
    return bit

In [7]:
shots = 8192
def calculate_P():
    P = {}
    for hashes in ['VV', 'VH', 'HV', 'HH']:
        
        # calculate each P[hashes] by sampling over 'shots'
        P[hashes] = 0
        for shot in range(shots):
            
            A, B = setup_variables()
            
            # hash type for 'A' is the 1st letter of 'hashes'
            a = hash2bit(A, hashes[0])
            # hash type for 'A' is the 2nd letter of 'hashes'
            b = hash2bit(B, hashes[1])
            
            P[hashes] += (a != b)/shots
    
    return P

In [9]:
P = calculate_P()
print(P)

{'VV': 0.2540283203125, 'VH': 0.0, 'HV': 0.6229248046875, 'HH': 0.3709716796875}


In [10]:
def bell_test(P):
    sum_P = sum(P.values())
    for hashes in P:
        
        bound = sum_P - P[hashes]
        
        print("The upper bound for P['{}'] is {}.".format(hashes, bound))
        print("The value of P['{}'] is {}.".format(hashes, P[hashes]))
        if P[hashes] <= bound:
            print("The upper bound is obeyed :)\n")
        else:
            if P[hashes]-bound < 0.1:
                print("This seems to have gone over the upper bound, "
                      "but only by a little bit :S\nProbably just rounding"
                      " errors or statistical noise.\n")
            else:
                print("This has gone over the upper bound :(\n")

In [11]:
bell_test(P)

The upper bound for P['VV'] is 0.993896484375.
The value of P['VV'] is 0.2540283203125.
The upper bound is obeyed :)

The upper bound for P['VH'] is 1.2479248046875.
The value of P['VH'] is 0.0.
The upper bound is obeyed :)

The upper bound for P['HV'] is 0.625.
The value of P['HV'] is 0.6229248046875.
The upper bound is obeyed :)

The upper bound for P['HH'] is 0.876953125.
The value of P['HH'] is 0.3709716796875.
The upper bound is obeyed :)



# Bell test for quantum variables

In [17]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

def initialize_program():
    qubit = QuantumRegister(2)
    A = qubit[0]
    B = qubit[1]
    
    bit = ClassicalRegister(2)
    a = bit[0]
    b = bit[1]
    
    qc = QuantumCircuit(qubit, bit)
    
    return  A, B, a, b, qc

In [18]:
def hash2bit(variable, hash_type, bit, qc):
    if hash_type == 'H':
        qc.h(variable)
    qc.measure(variable, bit)

In [19]:
import numpy as np

puzzle_program = ['qc.h(A)','qc.cz(A,B)','qc.ry(np.pi/4,B)','qc.ry(np.pi/4,A)','qc.cz(A,B)','qc.x(A)','qc.cz(A,B)','qc.h(B)','qc.cz(A,B)','qc.ry(np.pi/4,B)','qc.cz(A,B)','qc.cz(A,B)','qc.x(B)','qc.x(A)','qc.ry(np.pi/4,B)','qc.ry(np.pi/4,A)','qc.cz(A,B)','qc.h(B)']

def setup_variables(A, B, qc):
    for line in puzzle_program:
        eval(line)

In [20]:
shots = 8192
from qiskit import assemble, transpile

def calculate_P(backend):
    P = {}
    program = {}
    for hashes in ['VV', 'VH', 'HV', 'HH']:
        
        A, B, a, b, program[hashes] = initialize_program()
        
        setup_variables(A, B, program[hashes])
        
        hash2bit(A, hashes[0], a, program[hashes])
        hash2bit(B, hashes[1], b, program[hashes])
        
    # submit jobs
    t_qcs = transpile(list(program.values()), backend)
    qobj = assemble(t_qcs, shots=shots)
    job = backend.run(qobj)
    
    # get results
    for hashes in ['VV', 'VH', 'HV', 'HH']:
        stats = job.result().get_counts(program[hashes])
        
        P[hashes] = 0
        for string in stats.keys():
            a = string[-1]
            b = string[-2]
            
            if a != b:
                P[hashes] += stats[string]/shots
                
    return P

In [21]:
from qiskit import Aer
backend = Aer.get_backend('aer_simulator')

In [22]:
P = calculate_P(backend)
print(P)

{'VV': 0.252197265625, 'VH': 0.424560546875, 'HV': 0.2484130859375, 'HH': 0.927978515625}


In [23]:
bell_test(P)

The upper bound for P['VV'] is 1.6009521484375.
The value of P['VV'] is 0.252197265625.
The upper bound is obeyed :)

The upper bound for P['VH'] is 1.4285888671875.
The value of P['VH'] is 0.424560546875.
The upper bound is obeyed :)

The upper bound for P['HV'] is 1.604736328125.
The value of P['HV'] is 0.2484130859375.
The upper bound is obeyed :)

The upper bound for P['HH'] is 0.9251708984375.
The value of P['HH'] is 0.927978515625.
This seems to have gone over the upper bound, but only by a little bit :S
Probably just rounding errors or statistical noise.

